# Project description

This project's goal is to analyze where free/open hotspots and road sections are located. As a result, we are able to pinpoint the areas and routes with the best internet infrastructure for testing new services. Koln will serve as my use case.
It will be simpler to choose the locations for new services, such as speed monitoring, traffic light smart atomation, or any IOT application related to smart cities, if you have access to the road sections with the appropriate internet infrastructure. This site will make it feasible to test new technologies without having to worry about building the necessary infrastructure.

In [66]:
import pandas as pd
import re
import math
import haversine as hs
from haversine import Unit


## Load data

We are not running the data pipeline here becasue our data has been updated , but we use a local version which i have downloaded beforehand. 

In [1]:
hotspots_in_koeln_df = pd.read_sql_table('hotspots_in_koeln', 'sqlite:///../data/project.sqlite')
road_sections_df = pd.read_sql_table('road_sections','sqlite:///../data/project.sqlite')

In [14]:
road_sections_df['Geometry_paths'][0]

'[[6.956258032574851, 50.93279687112151], [6.956549764216335, 50.93287852984841], [6.956672094910387, 50.932912765544394], [6.956909600700839, 50.93300682451176], [6.95696711239904, 50.93302963470507], [6.9571878396903895, 50.933117088507174], [6.957590857475961, 50.93326983710532], [6.957814318941027, 50.93335870549693], [6.958167414994204, 50.93340794445326], [6.95844853934104, 50.9334369188863], [6.959427856852263, 50.933610904461894], [6.959468693848826, 50.93361820985403]]'

# Data preparation

steps:

* making a standard street name. 

In [2]:
# Making standard street names to match with the street names from the roads table
hotspots_in_koeln_df['Straße_name'] = hotspots_in_koeln_df['Straßenname'].map(lambda x: re.split('[s|S]tr[ |a|"."]', x)[0])

In [3]:
# Making standard street names to match with the street names from the hotspot table
road_sections_df['Straße_name'] = road_sections_df['Straße'].map(lambda x: re.split('[s|S]tr[ |a|"."]', x)[0])

In [5]:
# Merging the hotspots and the road section tables
all_in_one = hotspots_in_koeln_df.merge(road_sections_df , on='Straße_name')
all_in_one['houses in the intersection'] = all_in_one[['Hausnummer von links' , 'Hausnummer bis links' ,'Hausnummer von rechts'	, 'Hausnummer bis rechts' ]].values.tolist()

# Algorithms and solution

## First solution : we filter the road section that has house number that directly have a hotspot

In [9]:
interesections_with_hotspots = []
for i , row in all_in_one.iterrows():
    try :
        # Check if the house number are already one of the houses in the intersection
        if row['Haus Nr'].zfill(4) in row['houses in the intersection']:
            interesections_with_hotspots.append(row)
    except:
        pass
interesections_with_hotspots_df = pd.DataFrame(interesections_with_hotspots)

# Seconed solution : we filter the road section that has house number that in a range of X meter around the intersection and has a hotspot.

* There are many different types of spatial references for defining geographic data. To simplify accessing and referencing them, they are commonly referred to by a well-known ID (WKID) — an integer value.
* Our data has germetrical/spatial refrence which is wkid: 4326. The corrdinates provided is latitude and longitude specified in degrees.
* To calculate the distance between two location we use Haversine Distance.
* Haversine Distance can be defined as the angular distance between two locations on the Earth’s surface.
* Usually the range of the hotspot wifi is 45 meter , so this would be our threshold

In [59]:
# Combine the geometry coordinates of the hotspot in one coloum
all_in_one['hotspot_geometry'] = all_in_one.apply(lambda x: list([x['Geometry_x'],x['Geometry_y']]) ,axis=1)  

In [116]:
# make the geomtry paths list of floats instead of string
all_in_one['Gemotry_path_list'] = all_in_one['Geometry_paths'].map(lambda x: [list(map(float , element.split(','))) for element in re.findall(r"[.^[]*\[([^]]*)\]", x)] )

In [119]:
interesections_with_range_of_hotspots = []
for i , row in all_in_one.iterrows():
    loc1 = row['hotspot_geometry']
    for loc2 in row['Gemotry_path_list']:
        # check if the one of the intersection paths is in the range of the hotspot
        if hs.haversine(loc1 , loc2,unit=Unit.METERS) < 45:
            interesections_with_range_of_hotspots.append(row)
            #print("in the range") 
interesections_with_range_of_hotspots_df = pd.DataFrame(interesections_with_range_of_hotspots)

# Visualization of the hotspot that has intersections in its range 